# Classify text with BERT

BERT and other Transformer encoder architectures have been wildly successful on a variety of NLP tasks. They compute vector space representations of natural language that are suitable for use in Deep Learning models. The BERT family of model uses the transformer encoder architecture to process each token of input text in the full context of tokens before and after.

## BERT models results

Model | Architecture | Loss | Accuracy | Training Time (5 epochs)
--- | --- | --- | --- | ---
Sm BERT | L-12_H-768_A-12/3 | 0.6232010126113892 | 0.8850799798965454 | ?
Albert | - | 0.6523738503456116 | 0.884880006313324 | 24 mins (1 GPU)


In [ ]:
#@title ##Install dependencies
#@markdown - tensorflow-text: text preprocessing
#@markdown - tf-models-official: pre-trained models hosted on tensorflow-hub
%%capture --no-stderr
!pip install -Uqq tensorflow-text
!pip install -Uqq tf-models-official

In [ ]:
#@title ##Import packages
import os
import shutil
import uuid

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from google.colab import auth, data_table
from official.nlp import optimization
from tensorflow.config.experimental import list_physical_devices
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, Input 
from tensorflow.keras.preprocessing import text_dataset_from_directory

tf.get_logger().setLevel("ERROR")

print("tensorflow:", tf.__version__)
print("tensorflow_hub:", hub.__version__)

In [ ]:
#@title ## Google Cloud Setup
#@markdown - Authenticate user
#@markdown - Project config
#@markdown - Create dataset bucket
#project_id = "gcp project id" #@param {type:"string"}

#auth.authenticate_user()

# !gcloud config set project {project_id}

# bucket_name = f"bert-text-cls-{uuid.uuid1()}"
# !gsutil mb gs://{bucket_name}

# with open("to_upload.txt", "w") as f:
#     f.write("my sample file")

# !gsutil cp to_upload.txt gs://{bucket_name}
# !gsutil cat gs://{bucket_name}/to_upload.txt
# !gsutil rm -f -r gs://{bucket_name}

In [ ]:
#@title Choose Accelerator Strategy
accelerator_strategy = "GPU" #@param ["TPU", "GPU", "None"]

if accelerator_strategy == "TPU":
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
        print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    except ValueError:
        raise BaseException('ERROR: Not connected to a TPU runtime.')

    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("TPU devices: ", tf.config.list_logical_devices('TPU'))
else:
    gpus = list_physical_devices('GPU')
    if gpus:
        # Restrict TensorFlow to only use the first GPU
        try:
            tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print('Running on GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        except RuntimeError as e:
            # Visible devices must be set before GPUs have been initialized
            print(e)
            print("Num GPUs available:", len(list_physical_devices(GPU)))
    else:
        print("Running on CPU.")

In [ ]:
#@title Download Dataset - Large Movie Review Dataset
#@markdown [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/) 
#@markdown that contains the text of 50,000 movie reviews from the 
#@markdown [Internet Movie Database](https://www.imdb.com/).

url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset = tf.keras.utils.get_file(
    "aclImdb_v1.tar.gz", url, untar=True, cache_dir=".", cache_subdir=""
)
dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")
train_dir = os.path.join(dataset_dir, "train")
print("Train dataset:", train_dir)

# remove unused folders from train folder
remove_dir = os.path.join(train_dir, "unsup")
shutil.rmtree(remove_dir)


The IMDB dataset has already been divided into train and test, but it lacks a validation set. Let's create a validation set using an 80:20 split of the training data by using `validation_split`. 

>When using `validation_split` and `subset` or to pass `suffle=False` so that the validation and training splits have no overlap.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 32
seed = 42

def optimize_dataset(dataset):
    return dataset.cache().prefetch(buffer_size=AUTOTUNE)

raw_train_ds = text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

class_names = raw_train_ds.class_names
train_ds = optimize_dataset(raw_train_ds)

val_ds = text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)
val_ds = optimize_dataset(val_ds)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)
test_ds = optimize_dataset(test_ds)

In [ ]:
text_batch, label_batch = next(train_ds.as_numpy_iterator())
df = pd.DataFrame({"text": text_batch, "label": label_batch})
data_table.DataTable(df, include_index=False, num_rows_per_page=3, )

## BERT models overview

Currently these is the family of BERT models available on Tensorflow Hub:

- `BERT-Base`, `Uncased` the original BERT models.
- `Small BERTs` maintain the original architecture but with fewer and/or smaller Transformer blocks.
- `ALBERT` reduces the model size by sharing parameters between layers. Doesn't improve processing times.
- `BERT Experts` offer a choice of domain specific pre-trained models.
- `Electra` gets trained as a discriminator from GANs. (A must try!)
- `BERT with talking-heads Attention` has improved the core of the Transformers architecture.

### Game plan

1. Start with a smaller model (`sm BERT uncased`) since they are faster to train.
2. Upgrade to ALBERT looking for higher accuracy.
3. Models like Electra, Talking Heads or BERT expert are the next options in terms of improving accuracy.




In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = "sm electra" #@param ["sm BERT uncased", "albert", "sm electra", "talking heads"]

map_name_to_handle = {
    'sm BERT uncased':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'albert':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'sm electra':
        'https://tfhub.dev/google/electra_small/2',
    'talking heads':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'sm BERT uncased':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'albert':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/2',
    'sm electra':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'talking heads':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

## Explore the pre-processing layer



In [ ]:
bert_preprocessing_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
test_sentence = ['a masterpiece, maybe the best movie of the year!']
preprocessed_sentence = bert_preprocessing_model(test_sentence)

print(f"Keys       : {list(preprocessed_sentence.keys())}")
#@markdown `input sequence` length is truncated to 128 tokens.<br><br>
print(f"Shape      : {preprocessed_sentence['input_word_ids'].shape}")
print(f"Word Ids   : {preprocessed_sentence['input_word_ids'][0, :5]}")
#@markdown `input_type_ids` only have zeros because this is a single sentence 
#@markdown input. For a multiple sentence input, it would have one number for 
#@markdown each input.
print(f"Input Mask : {preprocessed_sentence['input_mask'][0, :5]}")
print(f"Type Ids   : {preprocessed_sentence['input_type_ids'][0, :5]}")

## Explore the encoder layer

In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
bert_results = bert_model(preprocessed_sentence)

print(f"Loaded BERT          : {tfhub_handle_encoder}")
print(f"Model keys           : {list(bert_results.keys())}")
#@markdown `pooled_output` represents each input sentence as a whole. It's 
#@markdown equivalent to the embedding for the entire sentence.<br><br>
print(f"Pooled outputs shape : {bert_results['pooled_output'].shape}")
print(f"Pooled outputs values: {bert_results['pooled_output'][0,:5]}")

#@markdown `sequence_output` represents each input token in the context.
#@markdown Equivalent to the contextual embedding for every token in the 
#@markdown sentence.<br><br>
print(f"Sequence outputs shape : {bert_results['sequence_output'].shape}")
print(f"Pooled outputs values: {bert_results['sequence_output'][0,:5]}")

#@markdown `encoder_outputs` are the intermediate activations of the `L` 
#@markdown transformer blocks.<br><br>

#@markdown For fine tunning we will use the `pooled_output` array.<br><br>

## Build the classifier model

In [ ]:
def build_classifier_model(optimizer, loss, metrics):
    text_input = Input(shape=(), dtype=tf.string, name="text")
    preprocessing_layer = hub.KerasLayer(
        tfhub_handle_preprocess, name="preproc"
    )
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(
        tfhub_handle_encoder, trainable=True, name="encoder"
    )
    outputs = encoder(encoder_inputs)
    net = outputs["pooled_output"]
    net = Dropout(0.1)(net)
    net = Dense(1, activation=None, name="classifier")(net)

    model = Model(text_input, net)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

### Build model using acceleration scope

In [ ]:
#@title ### Loss function
#@markdown BinaryCrossEntropy from Logits.<br>
#@markdown Accuracy metric: Binary accuracy.
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

### Optimizer 
For fine-tunning, let's use the same optimizer the BERT was originally trained with: Adaptative Moments(ADAM). This optimizer minimizes the prediction loss and does regularization by weight decay(nout using moments), which is also known as AdamW.

For the learning rate `init_lr`, we use the same schedule as BERT during pretraining: linear decay of a nontional initial learning rate, prefixed with a linear warm-up phase of the 10% of training steps `num_warmup_steps`. In line with BERT, the initial learning rate is smaller for fine-tunning (best of 5e-5, 3e-5, 2e-5). 

In [ ]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1 * num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(
    init_lr=init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type="adamw"
)

In [ ]:
classifier_model = build_classifier_model(optimizer, loss, metrics)

Let's test the our preprocessing layer integration

In [ ]:
bert_raw_result = classifier_model(tf.constant(test_sentence))
print(tf.sigmoid(bert_raw_result).numpy()[0])

Let's take a look at the model's structure.

In [ ]:
tf.keras.utils.plot_model(classifier_model)

## Model Training

In [ ]:
print(f"Training model with {tfhub_handle_encoder}")
history = classifier_model.fit(
    x=train_ds,
    validation_data=val_ds,
    epochs=epochs
)

## Evaluate model

Let's see how the model performs. Two values will be returned. Loss (a number which represents the error, lower values are better), and accuracy.

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
#@title ## Plot accuracy and loss
history_dict = history.history
# print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right');

## Export Model for Inference

In [ ]:
dataset_name = "imdb"
dataset_name = dataset_name.replace("/", "_")
saved_model_path = f"./{dataset_name}_bert"
classifier_model.save(saved_model_path, include_optimizer=False)
print("Saved model:", saved_model_path)

Let's reload the model, so we can try it side by side with the model that is still in memory.

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

Let's validate the `reloaded_model` accuracy.

In [ ]:
def print_samples(inputs, results):
    result_for_printing = [
        f"input: {inputs[i]:<30}  score: {results[i][0]:.6f}"
        for i in range(len(inputs))
    ]
    print(*result_for_printing, sep="\n")
    print()


examples = [
    "this is such an amazing movie!",
    "The movie was great!",
    "The movie was meh.",
    "The movie was okish.",
    "The movie was terrible...",
]

reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))
original_results = tf.sigmoid(classifier_model(tf.constant(examples)))

print("Results from the <saved> model:")
print_samples(examples, reloaded_results)

print("Results from the <original> model:")
print_samples(examples, original_results)

## Production ready model

In order to serve the saved model on TF Serving, we need to get the results througn the named signatures (`serving_default` in our case).

In [ ]:
serving_results = reloaded_model.\
    signatures["serving_default"](tf.constant(examples))
serving_results = tf.sigmoid(serving_results["classifier"])
print_samples(examples, serving_results)
